In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import numpy as np
import tools, RNTK_avg
import jax
import symjax
import symjax.tensor as T
import copy
from RNTK_NEW import RNTK

In [209]:
Ti = 1
TiPrime = 1

In [118]:
dataset = "trains"

In [119]:
dic = {k:v for k,v in map(lambda x : x.split(), open("data" + "/" + dataset + "/" + dataset + ".txt", "r").readlines())}

In [213]:
# N = int(dic["n_patrons1="])
# length = int(dic["n_entradas="])
N = 1000
length = 100

In [214]:
DATA = T.Placeholder((N, length), 'float32', name = "X")
x = DATA[:,0]
X = x*x[:, None]
n = X.shape[0]

## TODO: define global variable and use inside of symjax for X placeholders
## TODO: 



In [215]:
rntk = RNTK(dic, Ti, TiPrime, X, n) #could be flipped

## LAYER ZERO ATTEMPT


In [216]:
tiprimes, tis = rntk.get_diag_indices()
jtiprimes, jtis = rntk.get_diag_indices(jnpbool = True)

1 0 1
1 0 1


## perform diagonal in parallel and scan in parallel

In [14]:
# import jax.numpy as jnp

In [33]:
# def sequential_diags(carry, idx, tiprimesph, tisph):
#     print(tiprimesph[carry])
#     print(carry)
#     return carry+1, rntk.diag_func_wrapper(tiprimesph[idx],tisph[idx], jmode = False)

In [37]:
diagtest = T.Placeholder((rntk.dim_num, ), "int32")
diagtest

Placeholder(name=unnamed_24, shape=(5,), dtype=int32, scope=/default)

In [39]:
all_ema = T.scan(sequential_diags, init = 0, sequences=[diagtest], non_sequences=[jtiprimes, jtis])

Op(name=_rewriting_take_104, fn=_rewriting_take, shape=(), dtype=int32, scope=/default)
Placeholder(name=_68, shape=(), dtype=int32, scope=/default)
dim1idx Op(name=_rewriting_take_105, fn=_rewriting_take, shape=(), dtype=int32, scope=/default)
test Op(name=subtract_62, fn=subtract, shape=(), dtype=int32, scope=/default)


TypeError: Shapes must be 1D sequences of concrete values of integer type, got (Op(name=subtract_62, fn=subtract, shape=(), dtype=int32, scope=/default),).

In [ ]:
# all_ema

Op(name=_scan[1], fn=_scan, shape=(5,), dtype=int32, scope=/default)

## perform all in one scan

In [217]:
from RNTK_onediag import RNTK as RNTKOD
rntkod = RNTKOD(dic, Ti, TiPrime, X, n) #could be flipped

In [218]:
final_ema = rntkod.create_func_for_diag()

In [219]:
rntkod.no_bc_arrays_to_diag(final_ema)

(array([[Op(name=_rewriting_take_1191, fn=_rewriting_take, shape=(1000, 1000), dtype=float32, scope=/default),
         Op(name=_rewriting_take_1193, fn=_rewriting_take, shape=(1000, 1000), dtype=float32, scope=/default)],
        [Op(name=_rewriting_take_1195, fn=_rewriting_take, shape=(1000, 1000), dtype=float32, scope=/default),
         Op(name=_rewriting_take_1197, fn=_rewriting_take, shape=(1000, 1000), dtype=float32, scope=/default)]],
       dtype=object),
 array([[Op(name=_rewriting_take_1192, fn=_rewriting_take, shape=(1000, 1000), dtype=float32, scope=/default),
         Op(name=_rewriting_take_1194, fn=_rewriting_take, shape=(1000, 1000), dtype=float32, scope=/default)],
        [Op(name=_rewriting_take_1196, fn=_rewriting_take, shape=(1000, 1000), dtype=float32, scope=/default),
         Op(name=_rewriting_take_1198, fn=_rewriting_take, shape=(1000, 1000), dtype=float32, scope=/default)]],
       dtype=object))

## perform diagonal in parallel but scan sequentially

In [134]:
array_of_diags=T.concatenate([rntk.diag_func_wrapper(i[0],i[1]) for i in zip(tiprimes, tis)])

Op(name=concatenate_333, fn=concatenate, shape=(1, 2, 10, 10), dtype=float32, scope=/default)
Op(name=concatenate_336, fn=concatenate, shape=(2, 2, 10, 10), dtype=float32, scope=/default)
Op(name=concatenate_339, fn=concatenate, shape=(3, 2, 10, 10), dtype=float32, scope=/default)
Op(name=concatenate_342, fn=concatenate, shape=(2, 2, 10, 10), dtype=float32, scope=/default)
Op(name=concatenate_345, fn=concatenate, shape=(1, 2, 10, 10), dtype=float32, scope=/default)


In [139]:
test = T.Placeholder((2,10,10), dtype="float32")

In [135]:
full_lambda, full_phi = rntk.arrays_to_diag(array_of_diags)

[0, 1.0, 3.0, 6.0, 8.0]


In [ ]:
rntk.dim_lengths

array([1., 2., 3., 4., 3., 2., 1.])

In [ ]:
print("LAMBDA")
print(full_lambda)
print("PHI")
print(full_phi)

LAMBDA
[[Op(name=_rewriting_take_772, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)
  Op(name=_rewriting_take_776, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)
  Op(name=_rewriting_take_780, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)
  Op(name=_rewriting_take_784, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)]
 [Op(name=_rewriting_take_788, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)
  Op(name=_rewriting_take_792, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)
  Op(name=_rewriting_take_796, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)
  Op(name=_rewriting_take_800, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)]
 [Op(name=_rewriting_take_804, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)
  Op(name=_rewriting_take_808, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)
  Op(name=_

## testing

In [ ]:
bc = rntk.sh ** 2 * rntk.sw ** 2 * T.eye(rntk.n, rntk.n) + (rntk.su ** 2)*X + rntk.sb ** 2 ## took out an X
single_boundary_condition = T.expand_dims(bc, axis = 0)
single_boundary_condition

Op(name=expand_dims_21, fn=expand_dims, shape=(1, 10, 10), dtype=float32, scope=/default)

In [ ]:
bc

Op(name=add_31, fn=add, shape=(10, 10), dtype=float32, scope=/default)

In [ ]:
bc = rntk.sh ** 2 * rntk.sw ** 2 * T.eye(rntk.n, rntk.n) + (rntk.su ** 2) + rntk.sb ** 2 ## took out an X
# single_boundary_condition = T.expand_dims(bc, axis = 0)
single_boundary_condition = T.expand_dims(T.Variable((bc), "float32", "boundary_condition"), axis = 0)

In [ ]:
single_boundary_condition

Op(name=expand_dims_20, fn=expand_dims, shape=(1, 10, 10), dtype=float32, scope=/default)

In [ ]:
T.Placeholder((bc), "float32")

KeyboardInterrupt: 

In [ ]:
single_boundary_condition = T.expand_dims(T.Placeholder((bc), "float32", "boundary_condition"), axis = 0)

KeyboardInterrupt: 

In [ ]:
single_boundary_condition